In [21]:
import pandas as pd

In [1]:
from datasets import load_dataset

dataset = load_dataset("tatsu-lab/alpaca", split="train[:1000]")


In [2]:
def format_instruction(example):
    instruction = f"### Instruction:\n{example['instruction']}\n"
    input_text = f"### Input:\n{example['input']}\n" if example['input'] else ""
    response = f"### Response:\n{example['output']}"
    return {"text": instruction + input_text + response}

dataset = dataset.map(format_instruction)


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [3]:
dataset[0]

{'instruction': 'Give three tips for staying healthy.',
 'input': '',
 'output': '1.Eat a balanced diet and make sure to include plenty of fruits and vegetables. \n2. Exercise regularly to keep your body active and strong. \n3. Get enough sleep and maintain a consistent sleep schedule.',
 'text': '### Instruction:\nGive three tips for staying healthy.\n### Response:\n1.Eat a balanced diet and make sure to include plenty of fruits and vegetables. \n2. Exercise regularly to keep your body active and strong. \n3. Get enough sleep and maintain a consistent sleep schedule.'}

In [4]:
pip install --upgrade torch transformers


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [5]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "EleutherAI/gpt-neo-125M"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)



A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.3.1 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/runpy.py", line 198, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/runpy.py", line 88, in _run_code
    exec(code, run_globals)
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/py

In [6]:
model

GPTNeoForCausalLM(
  (transformer): GPTNeoModel(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(2048, 768)
    (drop): Dropout(p=0.0, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPTNeoBlock(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPTNeoAttention(
          (attention): GPTNeoSelfAttention(
            (attn_dropout): Dropout(p=0.0, inplace=False)
            (resid_dropout): Dropout(p=0.0, inplace=False)
            (k_proj): Linear(in_features=768, out_features=768, bias=False)
            (v_proj): Linear(in_features=768, out_features=768, bias=False)
            (q_proj): Linear(in_features=768, out_features=768, bias=False)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPTNeoMLP(
          (c_fc): Linear(in_features=768, out_features=3072, bias=True)
          (c_proj): Linear(in_fe

In [7]:
tokenizer.pad_token = tokenizer.eos_token

In [8]:
def tokenize(example):
    return tokenizer(example["text"], truncation=True, padding="max_length", max_length=256)

tokenized_dataset = dataset.map(tokenize, batched=True, remove_columns=dataset.column_names)


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [9]:
from transformers import Trainer, TrainingArguments, DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

training_args = TrainingArguments(
    output_dir="./gpt-neo-alpaca-10k",
    per_device_train_batch_size=2,     # Keep this small for memory
    gradient_accumulation_steps=4,     # Accumulate to simulate a batch of 8
    num_train_epochs=2,
    logging_steps=50,
    save_steps=1000,
    save_total_limit=1,
    fp16=False,  # Not for MPS (Mac), only use on CUDA
    report_to="none",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator=data_collator,
)


In [10]:
trainer.train()

Step,Training Loss
50,2.233800
100,2.105200
150,1.935700
200,1.781200
250,1.784100


TrainOutput(global_step=250, training_loss=1.9680098571777345, metrics={'train_runtime': 608.7171, 'train_samples_per_second': 3.286, 'train_steps_per_second': 0.411, 'total_flos': 261207097344000.0, 'train_loss': 1.9680098571777345, 'epoch': 2.0})

In [18]:
model.save_pretrained("./gpt-neo-finetuned")
tokenizer.save_pretrained("./gpt-neo-finetuned")


('./gpt-neo-finetuned/tokenizer_config.json',
 './gpt-neo-finetuned/special_tokens_map.json',
 './gpt-neo-finetuned/vocab.json',
 './gpt-neo-finetuned/merges.txt',
 './gpt-neo-finetuned/added_tokens.json',
 './gpt-neo-finetuned/tokenizer.json')

In [19]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("./gpt-neo-finetuned")
model = AutoModelForCausalLM.from_pretrained("./gpt-neo-finetuned")


In [20]:
prompt = """### Instruction:
Give tips for healthy skin .
### Response:"""

# Move tokenizer input to CPU
inputs = tokenizer(prompt, return_tensors="pt").to("cpu")

# Move model to CPU
model.to("cpu")

# Generate output
outputs = model.generate(
    input_ids=inputs["input_ids"],
    attention_mask=inputs["attention_mask"],
    max_length=100,
    do_sample=True,
    temperature=0.7,
    top_p=0.9,
    top_k=50,
    pad_token_id=tokenizer.eos_token_id,
)

# Decode and print result
print(tokenizer.decode(outputs[0], skip_special_tokens=True))


### Instruction:
Give tips for healthy skin.
### Response:
1. Protect your skin from light, heat, and other harmful substances.
2. Make sure your skin is hydrated with hydrated oils and sun protection.
3. Have a sun-drenched bath.
4. Have a good and warm bath.
5. Use moisturizer and oil-based products.
6. Get a good skin routine.
7. Make sure to wash your hands and


In [22]:
# Load tokenizer (same for both)
tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neo-125M")

# Load base model
base_model = AutoModelForCausalLM.from_pretrained("EleutherAI/gpt-neo-125M").to("cpu")

# Load fine-tuned model
finetuned_model = AutoModelForCausalLM.from_pretrained("./gpt-neo-finetuned").to("cpu")


In [23]:
def generate_response(model, prompt, tokenizer, max_length=100):
    inputs = tokenizer(prompt, return_tensors="pt").to("cpu")
    outputs = model.generate(
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        max_length=max_length,
        do_sample=True,
        temperature=0.7,
        top_p=0.9,
        top_k=50,
        pad_token_id=tokenizer.eos_token_id,
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True).replace(prompt, "").strip()


In [42]:
prompts = [
    "### Instruction:\nGive tips to stay healthy.\n### Response:",
    "### Instruction:\nPrepare me a plan to mountains\n### Response:",
    "### Instruction:\nWhat are the benefits of drinking water?\n### Response:",
    "### Instruction:\nSuggest ways to reduce screen time.\n### Response:",
    "### Instruction:\nWhat should I do if I feel anxious?\n### Response:"
]


In [43]:
data = []

for prompt in prompts:
    base_output = generate_response(base_model, prompt, tokenizer)
    finetuned_output = generate_response(finetuned_model, prompt, tokenizer)
    data.append([prompt, base_output, finetuned_output])

df = pd.DataFrame(data, columns=["Prompt", "Base Model", "Fine-Tuned Model"])


In [44]:
from IPython.display import display, HTML

def show_full_dataframe(df):
    styles = [
        dict(selector="td", props=[("white-space", "pre-wrap")]),
        dict(selector="th", props=[("white-space", "pre-wrap")]),
    ]
    html = df.style.set_table_styles(styles).to_html()
    display(HTML(html))


In [45]:
show_full_dataframe(df)

,Prompt,Base Model,Fine-Tuned Model
0,### Instruction: Give tips to stay healthy. ### Response:,"Don't give up! ## Chapter 5 What Happens When You're Not in the Heart of the Universe? ### What Happens When You're in the Heart of the Universe? In the last chapter, we've discussed some of the most important things about the universe and how we live it. We've also covered some of the ways in which we have to be careful about how",1. Eat lots of fruits and vegetables. 2. Try to stay hydrated and stay hydrated while exercising. 3. Eat enough to keep you healthy. 4. Exercise regularly. 5. Eat lots of healthy food. 6. Take a good nap. 7. Have a good meal. 8. Drink plenty of water. 9. Try to stay hydrated and hydrate while
1,### Instruction: Prepare me a plan to mountains ### Response:,Thank you for your reply. ### Question: What's the purpose of the instruction? ### Response: I know that the instructions are to get a mountain from a specific place. ### Question: What's the purpose of the instruction? ### Response: I know that the instructions are to get a mountain from a specific place. ### Question: What's,1. Write a plan for the mountains. 2. Plan out the terrain. 3. Take a trip to the top of the mountain. 4. Go to the summit of the mountain. 5. Follow a trail to the summit. 6. Take a short hike to the summit. 7. Go back to your car. 8. Return to the car. 9. Take a
2,### Instruction: What are the benefits of drinking water? ### Response:,"Thank you for your answer. It is quite a bit easier to find a good place to drink water in the UK than in the US. I would prefer that you do so. I don't think you have to be a water purifier, but I would advise that you do so. ------ dabren There's a couple of things I've noticed about the","Water is essential to life. It is a source of nutrients and is essential for the growth and development of plants. It also helps to regulate body temperature and reduce blood pressure. It also helps to control the blood pressure, which can lead to a variety of health issues. Drinking water can also help to prevent diseases such as diabetes and stroke. ### Response: Water can also help to reduce the risk of diseases"
3,### Instruction: Suggest ways to reduce screen time. ### Response:,The following are the responses from the current page.,- Implement a short text-to-speech system that utilizes a computer to monitor and analyze text. - Implement a short audio-to-visual system that utilizes a microphone to record audio. - Implement a short video-to-speech system that utilizes a voice recognition system to identify speech. - Implement a short text-to-speech system that utilizes a voice recognition system to identify text. - Implement
4,### Instruction: What should I do if I feel anxious? ### Response:,Thank you for your answers and I will try to answer some more questions. ### Suggested Answer: This is a response from a member of [ Lantern Forums] for which you have the option to reach the original consensus. ulture Follow [ Lantern Forums FAQ | FAQ |rish tocsis |rish tocsis.tumblr.com] on [Gizmodo |,1. Take a deep breath and focus on your goals. 2. Write a book or a blog post. 3. Get to know your audience and your products. 4. Create an online calendar to keep track of your favorite topics. 5. Write a blog post or blog post about your product. 6. Write a blog post about your customer service experience. 7. Send a
